In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [2]:
concrete_dataset = pd.read_csv('https://cocl.us/concrete_data')
concrete_dataset.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data_columns = concrete_dataset.columns

predictors = concrete_dataset[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_dataset['Strength'] # Strength column

In [19]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [20]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

In [21]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [22]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [23]:
model = regression_model()

In [24]:
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 262us/step - loss: 1549.4572
Epoch 2/50
721/721 [==============================] - 0s 85us/step - loss: 1535.1622
Epoch 3/50
721/721 [==============================] - 0s 84us/step - loss: 1520.5816
Epoch 4/50
721/721 [==============================] - 0s 83us/step - loss: 1504.9961
Epoch 5/50
721/721 [==============================] - 0s 77us/step - loss: 1488.1100
Epoch 6/50
721/721 [==============================] - 0s 57us/step - loss: 1469.6659
Epoch 7/50
721/721 [==============================] - 0s 56us/step - loss: 1449.4600
Epoch 8/50
721/721 [==============================] - 0s 54us/step - loss: 1427.4806
Epoch 9/50
721/721 [==============================] - 0s 56us/step - loss: 1403.6645
Epoch 10/50
721/721 [==============================] - 0s 52us/step - loss: 1377.5069
Epoch 11/50
721/721 [==============================] - 0s 54us/step - loss: 1349.8812
Epoch 12/50
721/721 [==============================] - 0s 56us

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [26]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 104us/step


251.4110738513539

In [27]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

251.41107581171372 0.0


In [28]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("Mean Sqr Err "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and std dev " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean Sqr Err 1: 30684.335709951458
Mean Sqr Err 2: 5316.27310881877
Mean Sqr Err 3: 1898.2897388248382
Mean Sqr Err 4: 749.9049741874621
Mean Sqr Err 5: 344.5534633401914
Mean Sqr Err 6: 240.53473325068896
Mean Sqr Err 7: 213.86212276717993
Mean Sqr Err 8: 138.82419001014486
Mean Sqr Err 9: 129.4856607875392
Mean Sqr Err 10: 115.46403345398147
Mean Sqr Err 11: 103.93495832523482
Mean Sqr Err 12: 100.7324509605235
Mean Sqr Err 13: 117.57747852609381
Mean Sqr Err 14: 124.20750397617377
Mean Sqr Err 15: 110.02967515963952
Mean Sqr Err 16: 111.85861465305958
Mean Sqr Err 17: 110.53150772045345
Mean Sqr Err 18: 96.33291811156042
Mean Sqr Err 19: 106.65099272990304
Mean Sqr Err 20: 128.9620449720463
Mean Sqr Err 21: 114.7692642952632
Mean Sqr Err 22: 107.78967774030075
Mean Sqr Err 23: 109.06058141714547
Mean Sqr Err 24: 103.9418918399749
Mean Sqr Err 25: 123.47289530436198
Mean Sqr Err 26: 100.02430497944162
Mean Sqr Err 27: 125.70855828936432
Mean Sqr Err 28: 115.99425193021213
Mean Sqr Er